In [0]:
import pandas as pd
import numpy as np

from itertools import chain
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,accuracy_score
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import GRU, MaxPooling1D, Embedding
from keras.models import Model
from keras import layers, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [4]:
!pip3 install hyperas

    100% |████████████████████████████████| 337kB 26.9MB/s 
ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.16
    Uninstalling prompt-toolkit-1.0.16:
      Successfully uninstalled prompt-toolkit-1.0.16


In [6]:
!pip3 install git+https://github.com/maxpumperla/hyperas.git

  Cloning https://github.com/maxpumperla/hyperas.git to /tmp/pip-req-build-ym0r1e2g
  Stored in directory: /tmp/pip-ephem-wheel-cache-k663ho3s/wheels/27/c7/75/b70097065b73570eda25350a796d87c41cd967471a04064cc2
Successfully built hyperas


In [7]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

ImportError: ignored

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = pd.read_csv('/content/gdrive/My Drive/NLP/Dataset/train_E6oV3lV.csv')
data.shape[0] - data.dropna().shape[0]
data_copy = data.copy()

### Tokenization

In [12]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [107]:
tokenized_single_posts = [nltk.tokenize.word_tokenize(i) for i in data.tweet]
len(tokenized_single_posts)


['@',
 'user',
 'when',
 'a',
 'father',
 'is',
 'dysfunctional',
 'and',
 'is',
 'so',
 'selfish',
 'he',
 'drags',
 'his',
 'kids',
 'into',
 'his',
 'dysfunction',
 '.',
 '#',
 'run']

In [111]:
print(tokenized_single_posts[0])

['@', 'user', 'when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', '.', '#', 'run']


In [0]:
leng = []
for i in range(len(tokenized_single_posts)):
    length = len(tokenized_single_posts[i])
    leng.append(length)

### stopwords removal

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import string
stp_removed = []
for i in range (len(tokenized_single_posts)):
    stp = [word for word in tokenized_single_posts[i] if word not in (stopwords.words('english')+list(string.punctuation))]
    stp_removed.append(stp)

In [113]:
print(len(stp_removed))
print(stp_removed[0])

31962
['user', 'father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction', 'run']


### Lemmatize the stop removed posts

In [20]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
words_lemma = []
lemma = nltk.WordNetLemmatizer()
for i in range(len(stp_removed)):
    words = [lemma.lemmatize(word) for word in stp_removed[i]]
    words_lemma.append(words)

In [116]:
print(len(words_lemma))
print(words_lemma[0])

31962
['user', 'father', 'dysfunctional', 'selfish', 'drag', 'kid', 'dysfunction', 'run']


### Remove all digit words

In [0]:
words_noNum = []
for i in range(len(words_lemma)):
    words = [word for word in words_lemma[i] if word.isdigit() == False]
    words_noNum.append(words)

In [119]:
len(words_noNum)
print(words_noNum[0])

['user', 'father', 'dysfunctional', 'selfish', 'drag', 'kid', 'dysfunction', 'run']


### Remove single character words

In [0]:
words_nonSingle = []
for i in range(len(words_noNum)):
    words = [word for word in words_noNum[i] if len(word) > 1]
    words_nonSingle.append(words)

In [121]:
len(words_nonSingle)

31962

### Remove non-alphabetic words

In [0]:
words_alpha = []
for i in range(len(words_nonSingle)):
    words = [word for word in words_nonSingle[i] if word.isalpha()]
    words_alpha.append(words)

In [124]:
print(len(words_alpha))
print(words_alpha[0])

31962
['user', 'father', 'dysfunctional', 'selfish', 'drag', 'kid', 'dysfunction', 'run']


In [0]:
data_copy['words_count'] = [len(i) for i in words_alpha]

In [33]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
noun_freq = []
verb_freq = []
adjective_freq = []
adverb_freq = []
for i in range(len(words_alpha)):
    word_pos_tag = nltk.pos_tag(words_alpha[i])
    count_noun = 0
    count_verb = 0
    count_adjective = 0
    count_adverb = 0
    for j in range(len(word_pos_tag)):
        if word_pos_tag[j][1] == "NN":
            count_noun += 1
        if word_pos_tag[j][1] == 'VB':
            count_verb += 1
        if word_pos_tag[j][1] == 'JJ':
            count_adjective += 1
        if word_pos_tag[j][1] == 'RB':
            count_adverb += 1
    noun_freq.append(count_noun/(len(words_alpha[i]) + 1))
    verb_freq.append(count_verb/(len(words_alpha[i])+1))
    adjective_freq.append(count_adjective/(len(words_alpha[i])+1))
    adverb_freq.append(count_adverb/(len(words_alpha[i])+1))

In [0]:
freq_dict = {'noun_freq' : noun_freq, 'verb_freq' : verb_freq, 'adjective_freq' : adjective_freq, 'adverb_freq' : adverb_freq}

In [0]:
data_copy = data_copy.join(pd.DataFrame(freq_dict))

In [141]:
data_copy = data_copy.drop(['id'],axis=1)
data_copy.tail()


,label,tweet,words_count,noun_freq,verb_freq,adjective_freq,adverb_freq
31957,0,ate @user isz that youuu?ðððððð...,4,0.600000,0.200000,0.000000,0.0
31958,0,to see nina turner on the airwaves trying to...,14,0.400000,0.066667,0.133333,0.0
31959,0,listening to sad songs on a monday morning otw...,8,0.666667,0.000000,0.111111,0.0
31960,1,"@user #sikh #temple vandalised in in #calgary,...",8,0.666667,0.000000,0.111111,0.0
31961,0,thank you @user for you follow,3,0.750000,0.000000,0.000000,0.0


In [143]:
features_distrbn = data_copy.groupby(['label'], as_index = False)['words_count', 'adjective_freq', 'noun_freq', 'adverb_freq', 'verb_freq'].mean()
np.round(features_distrbn, 3)

,label,words_count,adjective_freq,noun_freq,adverb_freq,verb_freq
0,0,7.785,0.162,0.446,0.050,0.032
1,1,8.368,0.180,0.436,0.049,0.025


#### read the GLoVE 100 file for preparing an Embedding layer of 100 dimenions

In [0]:
from google.colab import files
uploaded = files.upload()

In [144]:
MAX_SEQUENCE_LENGTH = 50
MAX_NUM_WORDS = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

embeddings_index = {}

with open("/content/glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


#### Initialize the Embedding layer with pre-trained weights from the GLoVE model:

In [0]:
## Define the sequence lengths, max number of words and embedding dimensions
MAX_SEQUENCE_LENGTH = 50   # Sequence length of each sentence. If more, crop. If less, pad with zeros
MAX_NB_WORDS = 10000        # Top 10000 frequently occuring words

#### Train-validation-test split. We separate validation data to use early stopping callback feature during training our model below

In [147]:
X = data_copy.iloc[:,1:7]
y = data_copy['label']
print(len(X),len(y))



31962 31962


In [0]:
x_train_val, x_test, y_train_val, y_test = train_test_split(X,y, test_size = 0.3, random_state = 123)

In [0]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, 
                                    y_train_val, test_size = 0.2, random_state = 123)

rebuild the full training data-frame to double-check that word count and POS tagging remain useful features across the training data set as well. Note above when we analysed the test data we did not include any labels data or standardization measures, thereby strictly ensuring no leakage of test information into the trainnig space

In [150]:
train_full = pd.concat([x_train, y_train], axis = 1)
train_full.head()

,tweet,words_count,noun_freq,verb_freq,adjective_freq,adverb_freq,label
4574,here's a ?for the day the government and state...,12,0.692308,0.0,0.000000,0.000000,1
6142,cardiff!!!! ó¾©. #cardiff #euro2016 #euro #fo...,9,0.800000,0.0,0.000000,0.000000,0
22578,@user where on eah is pastor #mmusimaimane? is...,10,0.636364,0.0,0.181818,0.090909,1
3709,@user and the panel is seated! @user @user @u...,8,0.444444,0.0,0.222222,0.000000,0
5766,so often we think our country has come so far ...,9,0.300000,0.0,0.100000,0.200000,0


In [151]:
word_count_distrbn_train = train_full.groupby(['label'], 
        as_index = False)['words_count', 'adjective_freq', 'noun_freq', 'adverb_freq', 'verb_freq'].mean()
np.round(word_count_distrbn_train, 3)

,label,words_count,adjective_freq,noun_freq,adverb_freq,verb_freq
0,0,7.751,0.162,0.446,0.05,0.031
1,1,8.268,0.177,0.432,0.05,0.025


In [154]:
print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(17898, 6) (4475, 6) (9589, 6)
(17898,) (4475,) (9589,)


The train-validation-test original texts are tokenized and padded for machine-readable formatting

In [157]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)   # get the frequently occuring words
tokenizer.fit_on_texts(x_train.tweet)           
train_sequences = tokenizer.texts_to_sequences(x_train.tweet)
val_sequences = tokenizer.texts_to_sequences(x_val.tweet)
test_sequences = tokenizer.texts_to_sequences(x_test.tweet)

word_index = tokenizer.word_index               # dictionary containing words and their index
#print(tokenizer.word_index)                   # print to check
print('Found %s unique tokens.' % len(word_index)) # total words in the corpus
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
val_data = pad_sequences(val_sequences, maxlen = MAX_SEQUENCE_LENGTH)# get only the top frequent words on train
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)   # get only the top frequent words on test

print(train_data[0])
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

Found 31519 unique tokens.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  944    4    9    2   19    2 1477    7 1087 1854   65 2520
 3200  666 3201  412 3629    8 1329   20]
(17898, 50)
(4475, 50)
(9589, 50)


standardize all the numeric features generated above by by fitting them on training data and using that to transform validation and test data respectively


In [0]:
scaleable_cols = ['words_count', 'adjective_freq', 'noun_freq', 'adverb_freq', 'verb_freq']

In [158]:
scaler_multicol = MinMaxScaler()
train_multicol_scaled = scaler_multicol.fit_transform(x_train[scaleable_cols])
val_multicol_scaled = scaler_multicol.fit_transform(x_val[scaleable_cols])
test_multicol_scaled = scaler_multicol.fit_transform(x_test[scaleable_cols])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [161]:
train_multicol_scaled[2]

array([0.27027027, 0.22727273, 0.65356265, 0.12121212, 0.        ])

In [162]:
train_data = np.hstack((train_data, train_multicol_scaled))
val_data = np.hstack((val_data, val_multicol_scaled))
test_data = np.hstack((test_data, test_multicol_scaled))

print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(17898, 55)
(4475, 55)
(9589, 55)


In [163]:
print(train_data[0])

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 9.44000000e+02 4.00000000e+00
 9.00000000e+00 2.00000000e+00 1.90000000e+01 2.00000000e+00
 1.47700000e+03 7.00000000e+00 1.08700000e+03 1.85400000e+03
 6.50000000e+01 2.52000000e+03 3.20000000e+03 6.66000000e+02
 3.20100000e+03 4.12000000e+02 3.62900000e+03 8.00000000e+00
 1.32900000e+03 2.00000000e+01 3.24324324e-01 0.00000000e+00
 7.11018711e-01 0.00000000e+00 0.00000000e+00]


initialize the embedding matrix from GLoVE 100 which will be used as the initial weights of the Embedding layer in our neural network

In [0]:
num_words = min(MAX_NB_WORDS, len(embeddings_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

The following cell declares our final model that has:




*   an initial Embedding layer with pre-trained weights from GLoVE 100 as described above, hence trainable = False at this layer,
*   two layers of GRU model with in-between BatchNormalization and Dropout as mentioned
*   the neural network uses the Keras function API and is then split after the second GRU layer into a one output model - 'label',
*   the output model has two Dense() layers as we want to visualize the learning of the network just prior to it's final predictions



In [172]:
posts_input = Input(shape=(None,), dtype='int32', name='all_posts')
embedded_posts = Embedding(input_dim=MAX_NB_WORDS,
                            input_length=MAX_SEQUENCE_LENGTH, 
                            output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)(posts_input)

x = layers.GRU(128, activation='relu', return_sequences = True)(embedded_posts)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.GRU(64, activation = 'relu')(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dropout(0.2)(x)

label_pred = layers.Dense(8, activation = 'relu', name = 'label0')(x)
label_pred = layers.Dropout(0.5)(label_pred)
label_pred = layers.Dense(1, activation = 'sigmoid', name = 'label1')(label_pred)


combined_model = Model(posts_input, [label_pred])
combined_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
all_posts (InputLayer)       (None, None)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 50, 100)           1000000   
_________________________________________________________________
gru_7 (GRU)                  (None, 50, 128)           87936     
_________________________________________________________________
batch_normalization_4 (Batch (None, 50, 128)           512       
_________________________________________________________________
dropout_10 (Dropout)         (None, 50, 128)           0         
_________________________________________________________________
gru_8 (GRU)                  (None, 64)                37056     
_________________________________________________________________
dense_4 (Dense)              (None, 16)                1040      
__________

In [0]:
callbacks_list = [EarlyStopping(monitor='val_loss', patience=1, ),
                ModelCheckpoint(filepath='model_multi-feature.h5', monitor='val_loss',
                        save_best_only=True,)]

In [0]:
combined_model.compile(optimizer = 'rmsprop',loss = {'label1' : 'binary_crossentropy'},metrics = ['acc'])

In [180]:
y_train.head()

4574     1
6142     0
22578    1
3709     0
5766     0
Name: label, dtype: int64

In [0]:
epochs = 16
batch_size = 32

In [183]:
hist = combined_model.fit(train_data, {'label1' : y_train},
                                  epochs = epochs, batch_size = batch_size,
                   callbacks = callbacks_list,
               validation_data = (val_data, {'label1' : y_val})).history

Instructions for updating:
Use tf.cast instead.
Train on 17898 samples, validate on 4475 samples
Epoch 1/16
17898/17898 [==============================] - 78s 4ms/step - loss: 0.2876 - acc: 0.9151 - val_loss: 0.1718 - val_acc: 0.9280
Epoch 2/16
17898/17898 [==============================] - 76s 4ms/step - loss: 0.2168 - acc: 0.9327 - val_loss: 0.1531 - val_acc: 0.9457
Epoch 3/16
17898/17898 [==============================] - 77s 4ms/step - loss: 0.1857 - acc: 0.9412 - val_loss: 0.2215 - val_acc: 0.9457


In [0]:
model1 = load_model('model_multi-feature.h5')

In [0]:
pred = model1.predict(test_data)

In [0]:
label_list = list(chain.from_iterable(pred))
label_predict = [1 if x >= 0.5 else 0 for x in label_list]
label_recall = recall_score(label_predict, y_test)
label_acc = accuracy_score(label_predict, y_test)

In [196]:
print("ACCURACY : ",label_acc)

print("Recall : ",label_recall)

ACCURACY :  0.9465011992908541
Recall :  0.7821428571428571
